# Autoencoders

In this Notebook we will be dealing (for most of the time) with the same Dataset as previously... but our task is different.

Instead of classifying the images we will be reducing their dimensions into so called "latent space" and then, reconstruct them into images again. Our goal is to obtain the output which will be similar to the input as much as possible...

Let's get familiar with a special group of Deep Neural Networks - Autoencoders!

- Autoencoders are a specific type of feedforward Neural Networks where the input is the same as the output

- They consist of 3 parts: **encoder**, **latent space** and **decoder**. 

- They compress the input into a lower-dimensional **latent space** (**encoder**) and then reconstruct the output from this representation (**decoder**).

- The role of basic Autoencoder is mostly dimensionality reduction. 

**TASK:** Train the simple Autoencoder model for 2 different types of data. 

1. Linear Autoencoder for MNIST dataset
2. Convolutional Autoencoder for MNIST dataset
3. **Exercise:** Build Autoencoder for real physics data!

**WHAT YOU WILL LEARN:**

- what are main components of Autoencoder
- how to implement Linear Autoencoder
- how to implement Convolutional Autoencoder
- how to add 1d and 2d Normalization layers to the model architecture
- how to build Dataset class for custom dataset
- how to implement Autoencoder for real data


**TO DO:** Read and understand the following code. Run the cells, analyse the results and if everything is clear, follow the instructions concerning exercises parts. 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torchvision.utils import make_grid 
import numpy as np
import pandas as pd

## 1. Autoencoder Linear

We will build the simple Linear Autoencoder to decode and encode popular dataset of images of handwritted digits. 

In order to test the efficiency of our algorithm we will compare orginal data with decoded data (visually).

### Data preprocessing 

Let's download our MNIST dataset and create DataLoader. In this case, we don't need two separated datasets (train and test), since we are dealing with unsupervised learning!

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),])

transform = transforms.ToTensor()

mnist_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

data_loader = torch.utils.data.DataLoader(dataset=mnist_data,
                                          batch_size=64,
                                          shuffle=True)



### Build the Autoencoder

In order to build an autoencoder we need 3 things: an encoding method, a decoding method, and a loss function to compare the output with the target. 

Both the encoder and decoder are fully-connected feedforward neural networks.

In [ ]:
# Defining Model

class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        # Role of encoder: repeatedly reduce the size
        # N - batch size
        # input dimensions: 28x28 = 784
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128), # (N, 784) -> (N, 128)
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 12),
            nn.ReLU(),
            nn.Linear(12, 3) # -> N, 3 - our Latent Space dimension is 3
        )
        # Role of decoder: repeatedly increase the size
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(),
            nn.Linear(12, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 28 * 28),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
model = Autoencoder()
print(model)

### Define Loss and Optimizer

Loss function: we use mean squared error (MSE).

In [ ]:
model = Autoencoder()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=1e-3, 
                             weight_decay=1e-5)

### Main training loop

We will train our Autoencoder with only 1 epoch!

In [ ]:
num_epochs = 1
outputs = []
for epoch in range(num_epochs):
    for (img, _) in data_loader:
        img = img.reshape(-1, 28*28) # we need to reshape 28x28 image into vector
        recon = model(img)
        loss = criterion(recon, img)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch:{epoch+1}, Loss:{loss.item():.4f}')
    outputs.append((epoch, img, recon))

### Compare original images with reconstructed images

In [ ]:
for k in range(0, num_epochs):
    plt.figure(figsize=(9, 2))
    plt.gray()
    imgs = outputs[k][1].detach().numpy()
    recon = outputs[k][2].detach().numpy()
    for i, item in enumerate(imgs):
        if i >= 9: break
        plt.subplot(2, 9, i+1)
        item = item.reshape(-1, 28,28) # -> use for Autoencoder_Linear
        # item: 1, 28, 28
        plt.imshow(item[0])

            
    for i, item in enumerate(recon):
        if i >= 9: break
        plt.subplot(2, 9, 9+i+1) # row_length + i + 1
        item = item.reshape(-1, 28,28) # -> use for Autoencoder_Linear
        # item: 1, 28, 28
        plt.imshow(item[0])

### **Exercise**

**Normalization layers**

Normalizing a set of data transforms the set of data to be on a similar scale. It may help to stabilize the process of training.
It is more useful for complex architectures with many layers, however let's learn how to add them in this simple example.
**Batch normalization** standardizes the input of a layer across a single batch.

Modify the architecture of the Autoencoder by adding `nn.BatchNorm1d(out)` layer after each Linear layer, where the `out` parameter should be equal to the size of the output of the previous Linear layer. 


In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        # Role of encoder: repeatedly reduce the size
        # N - batch size
        # input dimensions: 28x28 = 784
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128), # (N, 784) -> (N, 128)
            ## add BatchNorm1d layer here
            nn.ReLU(),
            nn.Linear(128, 64),
            ## add BatchNorm1d layer here
            nn.ReLU(),
            nn.Linear(64, 12),
            ## add BatchNorm1d layer here
            nn.ReLU(),
            nn.Linear(12, 3) # -> N, 3
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            ## add BatchNorm1d layer here
            nn.ReLU(),
            nn.Linear(12, 64),
            ## add BatchNorm1d layer here
            nn.ReLU(),
            nn.Linear(64, 128),
            ## add BatchNorm1d layer here
            nn.ReLU(),
            nn.Linear(128, 28 * 28),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

model = Autoencoder()
print(model)

####  Train your new architecture...

In [ ]:
model = Autoencoder()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=1e-3, 
                             weight_decay=1e-5)


num_epochs = 1
outputs = []
for epoch in range(num_epochs):
    for (img, _) in data_loader:
        img = img.reshape(-1, 28*28) # for Autoencoder_Linear we need to reshape 28x28 image into vector
        recon = model(img)
        loss = criterion(recon, img)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch:{epoch+1}, Loss:{loss.item():.4f}')
    outputs.append((epoch, img, recon))



for k in range(0, num_epochs):
    print(f'Epoch {k}:')
    plt.figure(figsize=(9, 2))
    plt.gray()
    imgs = outputs[k][1].detach().numpy()
    recon = outputs[k][2].detach().numpy()
    for i, item in enumerate(imgs):
        if i >= 9: break
        plt.subplot(2, 9, i+1)
        item = item.reshape(-1, 28,28) # -> use for Autoencoder_Linear
        # item: 1, 28, 28
        plt.imshow(item[0])

            
    for i, item in enumerate(recon):
        if i >= 9: break
        plt.subplot(2, 9, 9+i+1) # row_length + i + 1
        item = item.reshape(-1, 28,28) # -> use for Autoencoder_Linear
        # item: 1, 28, 28
        plt.imshow(item[0])

## 2. Autoencoder Conv

Since we are working with image data - let's use convolutions!

**Why convolutions?**

A convolutional layer is much more specialized and efficient, than a fully connected layer.

In a fully connected layer each neuron is connected to every neuron in the previous layer, and each connection has its own weight. A neuron in a convolutional layer, however, is only connected to "nearby" neurons from the preceding layer within the width of the convolutional kernel.


Typical use case for convolutional layers is for image data where, as required, the features are local (e.g. a "nose" consists of a set of nearby pixels, not spread all across the image).

The fewer number of connections and weights make convolutional layers relatively cheap (vs full connect) in terms of memory and compute power needed.

Read more about [components of convolution layers (kernel size, padding, stride)](https://medium.com/@RaghavPrabhu/understanding-of-convolutional-neural-network-cnn-deep-learning-99760835f148)

**TO DO:**
Run and analyse the following code with Autoencoder implemented with convolutional layers. Pay special attention to the dimensions of input and output of each layer (as always!).

**Conv vs Transposed Conv**

We will use 2 types of convolutional layers. 

1) `Conv2d` - A convolutional layer extracts features from the layer, and downsamples the input.


2) `ConvTranspose2d` - In a tranposed convolution, instead of the input being larger than the output, the output is larger. 
A transposed convolutional layer attempts to reconstruct the spatial dimensions of the convolutional layer and reverses the downsampling and upsampling techniques applied to it.

Read more about [transposed convolutions](https://medium.com/@marsxiang/convolutions-transposed-and-deconvolution-6430c358a5b6).


In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()        
        # input: N, 1, 28, 28
        # N - batch size
        # 1 - number of channels (RGB images would have 3 channels)
        # 28 x 28 - dimesions of input images
        # role of encoder: from N,1,28,28 -> N,64,1,1
        # Conv2d: in_channels, out_channels, ks, stride, padding
        # New dimensions (n x n) of each layer can be calculated from the formula: (n+2*pad - ks)//stride + 1
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=2, padding=1), # -> (28 + 2*1 - 3)//2 + 1 = 14x14 (ch: 1 -> 16) -> N, 16, 14, 14
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1), # -> (14+2*1 - 3)//2 + 1 = 7x7 (ch: 16 -> 32) -> N, 32, 7, 7
            nn.ReLU(),
            nn.Conv2d(32, 64, 7, stride=1, padding=0) # -> (7+2*0 - 7)//1 + 1 = 1x1 (ch: 32 -> 64) -> N, 64, 1, 1
        )
        
        # input: N, 64, 1, 1
        # N - batch size
        # 64 - number of channels
        # 1 x 1 - dimesions
        # role of encoder: from N,64,1,1 -> N,1,28,28 (back to original!)
        # ConvTranspose2d: in_channels, out_channels, ks, stride, padding, output_padding
        # New dimensions (n) of each layer can be calculated from the formula: (n-1)*stride - 2*padding + ks + out_padding
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 7, stride=1, padding=0), # -> (1-1)*1 - 2*0 + 7 = 7x7 (ch: 32 -> 64) -> N, 32, 7, 7
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1), # -> (7-1)*2 - 2*1 + 3 + 1 = 14x14 (ch: 32 -> 16) -> N, 16, 14, 14 
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1), # -> (14-1)*2 - 2*1 + 3 + 1 = 28x28 (ch: 16 -> 1) -> N, 1, 28, 28 
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
model = Autoencoder()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=1e-3, 
                             weight_decay=1e-5)

In [ ]:
num_epochs = 1
outputs = []
for epoch in range(num_epochs):
    for (img, _) in data_loader:
        recon = model(img)
        loss = criterion(recon, img)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch:{epoch+1}, Loss:{loss.item():.4f}')
    outputs.append((epoch, img, recon))

In [ ]:
for k in range(0, num_epochs):
    plt.figure(figsize=(9, 2))
    plt.gray()
    imgs = outputs[k][1].detach().numpy()
    recon = outputs[k][2].detach().numpy()
    for i, item in enumerate(imgs):
        if i >= 9: break
        plt.subplot(2, 9, i+1)
        # item: 1, 28, 28
        plt.imshow(item[0])
            
    for i, item in enumerate(recon):
        if i >= 9: break
        plt.subplot(2, 9, 9+i+1) # row_length + i + 1
        # item: 1, 28, 28
        plt.imshow(item[0])

### **Exercise**

1. Compare the results of Linear and Convolutional Autoencoders (visible results, loss value, time of training). You may experiment with a larger numbers of epoch for both.

2. Similarly to the example with Linear Autoencoder - add some Normalization layers. REMARK - use `nn.BatchNorm2d(out)` after each Conv2d or Conv2dTranspose layer.

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=2, padding=1), # -> N, 16, 14, 14
            ## add BatchNorm2d layer here
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1), # -> N, 32, 7, 7
            ## add BatchNorm2d layer here
            nn.ReLU(),
            nn.Conv2d(32, 64, 7, stride=1, padding=0), # -> N, 64, 1, 1
        )
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 7, stride=1, padding=0), #  -> N, 32, 7, 7
            ## add BatchNorm2d layer here
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1), # -> N, 16, 14, 14
            ## add BatchNorm2d layer here
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1), # -> N, 1, 28, 28 
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


model = Autoencoder()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=1e-3, 
                             weight_decay=1e-5)


num_epochs = 1
outputs = []
for epoch in range(num_epochs):
    for (img, _) in data_loader:
        recon = model(img)
        loss = criterion(recon, img)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch:{epoch+1}, Loss:{loss.item():.4f}')
    outputs.append((epoch, img, recon))


for k in range(0, num_epochs):
    plt.figure(figsize=(9, 2))
    plt.gray()
    imgs = outputs[k][1].detach().numpy()
    recon = outputs[k][2].detach().numpy()
    print("Original data")
    for i, item in enumerate(imgs):
        if i >= 9: break
        plt.subplot(2, 9, i+1)
        # item: 1, 28, 28
        plt.imshow(item[0])
    print("Reconstructed data")
    for i, item in enumerate(recon):
        if i >= 9: break
        plt.subplot(2, 9, 9+i+1) # row_length + i + 1
        # item: 1, 28, 28
        plt.imshow(item[0])

## 3. Build Autoencoder for real physics data!

Time for the main part of our Autoencoder Exercise! Let's try to use Autoencoder for reconstruction the data from Monte Carlo simulation of photon beam therapy (PBT). 

**About the data**

The data are so called phase spaces from Monte Carlo simulation of photon beam therapy (PBT). 
They contain information about dose delivery in a plane perpendicular to the axis of a beam of ionizing radiation. 

The phase space is a collection of particles (photons, electrons, and positrons) — for each particle its energy, momentum, and coordinates of the point of crossing the phase space plane are stored. 


Out data is collection of 995401 Photons. 
Photons in a phase space are described by six parameters: 
- position coordinates X, Y in the plane perpendicular to the beam axis
- direction of momentum (dX, dY, dZ) which is a unit vector
- energy (E) of the photon

In [ ]:
import torch
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from typing import List, Tuple, Type
import numpy as np
import pandas as pd
import copy
import random

CUDA_DEVICE_NUM=0
DEVICE = torch.device(f'cuda:{CUDA_DEVICE_NUM}' if torch.cuda.is_available() else 'cpu')
print('Device:', DEVICE)


In [ ]:
# Hyperparameters
RANDOM_SEED = 123
LEARNING_RATE = 0.0005
BATCH_SIZE = 64
NUM_EPOCHS = 10



### Prepare and show the data

In [ ]:
!wget https://csc-files.web.cern.ch/CSC2022/photons.npy

path = 'photons.npy'

photons = np.load(path)
X = np.zeros(photons.shape,dtype=np.float32)
np.copyto(X,photons)

df_data = pd.DataFrame(X, columns = ['X', 'Y', 'dX', 'dY', 'dZ', 'E'])
df_data.head()

In [ ]:
print(df_data.shape)

We will plot histograms of all parameters of particles: energy, momentum, and coordinates of the point of crossing the phase space plane.

In [ ]:
tmp=df_data.to_numpy(dtype=np.float32)
#print(tmp)
tmp=tmp[:,:]
orginal=copy.deepcopy(tmp)

fig, axs = plt.subplots(2, 3)
fig.set_size_inches(16,8)
bins = 100
axs[0, 0].hist(orginal[:,0],bins=bins, label ='orginal',alpha=0.5)
axs[0, 0].set_title('E')
axs[0, 1].hist(orginal[:,1],bins=bins, label ='orginal',alpha=0.5)
axs[0, 1].set_title('X')
axs[0, 2].hist(orginal[:,2],bins=bins, label ='orginal',alpha=0.5)
axs[0, 2].set_title('Y')
axs[1, 0].hist(orginal[:,3],bins=bins, label ='orginal',alpha=0.5)
axs[1, 0].set_title('dX')
axs[1, 1].hist(orginal[:,4],bins=bins, label ='orginal',alpha=0.5)
axs[1, 1].set_title('dY')
axs[1, 2].hist(orginal[:,5],bins=bins, label ='orginal',alpha=0.5)
axs[1, 2].set_title('dZ')

Let's build dedicated PhotonsDataset class that inherits from Dataset. From this point we will be able to create DataLoader.

In [ ]:
class PhotonsDataset(Dataset):
    def __init__(self, data, transform=None):
        self.x = data
        self.n_samples = len(data)
        self.transform = transform
        self.columns = ['X', 'Y', 'dX', 'dY', 'dZ', 'E']

    def __getitem__(self, index):
        sample = self.x

        if self.transform:
            sample = self.transform(sample)
        return sample[index]

    def __len__(self):
        return self.n_samples


class ToTensorFromNdarray:
    def __call__(self, sample):
        # Returns a tensor made from ndarray
        return torch.from_numpy(sample)

photons = np.load(path)
train_transforms = ToTensorFromNdarray()
train_dataset = PhotonsDataset(
        data=photons, transform=train_transforms)
train_loader = DataLoader(dataset=train_dataset,
                                  batch_size=BATCH_SIZE,
                                  num_workers=2,
                                  shuffle=True)



In [ ]:
print('Training Set:\n')
for features in train_loader:  
    print('Batch dimensions:', features.size())
    print(features[:4,:6])
    break

### **Exercise**: Build the Autoencoder

Try to design the Autoencoder that will be able to process our dataset with Photons. The rest (training loop, loss, optimizers) are already coded in next cells. Your role is to put layers into the encoder and decoder!

Hints:
1. Will you use Linear or Conv version of Autoencoder? 
2. What are input dimensions of our data?


In [ ]:
class Autoencoder_Linear(nn.Module):
    def __init__(self):
        super().__init__()        
        self.encoder = nn.Sequential(
            ### YOUR CODE HERE
        )
        
        self.decoder = nn.Sequential(
            ### YOUR CODE HERE
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded, encoded

In [ ]:
model = Autoencoder_Linear()
model.to(DEVICE)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=1e-3, 
                             weight_decay=1e-5)

In [ ]:
outputs = []
for epoch in range(NUM_EPOCHS):
    for features in train_loader:
        features = features.to(DEVICE)
        recon, _ = model(features)
        loss = criterion(recon, features)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch:{epoch+1}, Loss:{loss.item():.4f}')
    outputs.append((epoch, img, recon))

In [ ]:
tmp=df_data.to_numpy(dtype=np.float32)
tmp=tmp[:,:]
orginal=copy.deepcopy(tmp)
tmp=torch.from_numpy(tmp)
with torch.no_grad():
    result, result_encoded_features=model(tmp.to(device=DEVICE))
result=result.cpu().detach().numpy() 


Let's see how our model works by comparing histograms with reconstructed data!



In [ ]:
fig, axs = plt.subplots(2, 3)
fig.set_size_inches(16,8)
bins = 100
axs[0, 0].hist(orginal[:,0],bins=bins, label ='orginal',alpha=0.5)
axs[0, 0].hist(result[:,0],bins=bins, label ='encoded', alpha=0.5)
axs[0, 0].set_title('E')
axs[0, 1].hist(orginal[:,1],bins=bins, label ='orginal',alpha=0.5)
axs[0, 1].hist(result[:,1],bins=bins, label ='encoded', alpha=0.5)
axs[0, 1].set_title('X')
axs[0, 2].hist(orginal[:,2],bins=bins, label ='orginal',alpha=0.5)
axs[0, 2].hist(result[:,2],bins=bins, label ='encoded', alpha=0.5)
axs[0, 2].set_title('Y')
axs[1, 0].hist(orginal[:,3],bins=bins, label ='orginal',alpha=0.5)
axs[1, 0].hist(result[:,3],bins=bins, label ='encoded', alpha=0.5)
axs[1, 0].set_title('dX')
axs[1, 1].hist(orginal[:,4],bins=bins, label ='orginal',alpha=0.5)
axs[1, 1].hist(result[:,4],bins=bins, label ='encoded', alpha=0.5)
axs[1, 1].set_title('dY')
axs[1, 2].hist(orginal[:,5],bins=bins, label ='orginal',alpha=0.5)
axs[1, 2].hist(result[:,5],bins=bins, label ='encoded', alpha=0.5)
axs[1, 2].set_title('dZ')


fig.tight_layout()
fig.legend()

### **Exercise**

(Optional) Try to make your results better (experiment with deeper architecture, different dimension of latent space, hyperparameters).